In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In /Users/kuba/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/kuba/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/kuba/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/kuba/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/kuba/anaconda3/lib/python3.7/site-packages/matplotlib/

In [3]:
df = pd.read_csv("Data/cces_all_clean.csv")
df.head()

,abortion,state,eth,male,age,educ
0,1,MD,Other,-0.5,50-59,Some college
1,1,TN,White,-0.5,40-49,HS
2,1,OH,White,-0.5,60-69,Some college
3,0,CA,Other,-0.5,70+,Post-grad
4,1,KY,White,-0.5,40-49,HS


In [4]:
# separate between features/inputs (X) and target/output variables (y)
mat = df.values
X = mat[:,1:]
y = mat[:,0].astype("int")
ind = df.index.astype("int")
print(X)
print(X.shape)
print(y)
print(y.shape)
print(ind)
print(ind.shape)

[['MD' 'Other' -0.5 '50-59' 'Some college']
 ['TN' 'White' -0.5 '40-49' 'HS']
 ['OH' 'White' -0.5 '60-69' 'Some college']
 ...
 ['IN' 'White' -0.5 '18-29' 'Some college']
 ['CO' 'White' -0.5 '30-39' 'HS']
 ['GA' 'Other' 0.5 '30-39' 'Some college']]
(59810, 5)
[1 1 1 ... 0 0 0]
(59810,)
RangeIndex(start=0, stop=59810, step=1)
(59810,)


In [5]:
categories = ["state", "eth", "male", "age", "educ"]
for category in categories:
    df[category+"_cat"] = df[category].astype('category').cat.codes

In [6]:
df.dtypes

abortion       int64
state         object
eth           object
male         float64
age           object
educ          object
state_cat       int8
eth_cat         int8
male_cat        int8
age_cat         int8
educ_cat        int8
dtype: object

,abortion,state,eth,male,age,educ,state_cat,eth_cat,male_cat,age_cat,educ_cat
0,1,MD,Other,-0.5,50-59,Some college,19,2,0,3,4
2,1,OH,White,-0.5,60-69,Some college,34,3,0,4,4
7,1,AZ,White,-0.5,50-59,Some college,3,3,0,3,4
8,1,AL,White,0.5,40-49,Some college,1,3,1,2,4
9,1,GA,Black,-0.5,18-29,Some college,9,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...
59803,1,MO,Other,-0.5,18-29,Some college,23,2,0,0,4
59804,1,NV,Other,-0.5,18-29,Some college,32,2,0,0,4
59805,0,ME,White,-0.5,18-29,Some college,20,3,0,0,4
59807,0,IN,White,-0.5,18-29,Some college,14,3,0,0,4


In [7]:
mat = df.iloc[:,6:].values
X = mat
X = torch.tensor(X).float()
y = torch.tensor(y).float()
print(X.shape)

torch.Size([59810, 5])


In [8]:
from torch import sigmoid
def model(X, obs=None):
    
    sigmas = pyro.sample("sigmas", dist.HalfCauchy(torch.ones(8)*5.))
    gammas = pyro.sample("gammas", dist.Normal(torch.zeros(5), 5.0))
    alpha_educ = pyro.sample("alpha_educ", dist.Normal(torch.zeros(5), sigmas[0]))
    alpha_eth = pyro.sample("alpha_eth", dist.Normal(torch.zeros(4), sigmas[1]))
    alpha_age = pyro.sample("alpha_age", dist.Normal(torch.zeros(6), sigmas[2]))
    alpha_male_eth = pyro.sample("alpha_male_eth", dist.Normal(torch.zeros((2,4)), sigmas[3]))
    alpha_educ_age = pyro.sample("alpha_educ_age", dist.Normal(torch.zeros((5,6)), sigmas[4]))
    alpha_educ_eth = pyro.sample("alpha_educ_eth", dist.Normal(torch.zeros((5,4)), sigmas[5]))
    
    # need the GROUP LEVEL PREDICTORS data for states
    # Temporary solution
    ####################
    alpha_state = pyro.sample("alpha_state", dist.Normal(torch.zeros(50), sigmas[6]))
    ####################
    beta_male = pyro.sample("beta_male", dist.Normal(0.,5.))
    with pyro.plate("data", size=len(X), subsample_size=500) as ind:
        state = X[ind,0]
        eth = X[ind,1]
        male = X[ind,2]
        age = X[ind,3]
        educ = X[ind,4]
        y = pyro.sample("y",dist.Bernoulli(sigmoid(alpha_state[list(state)] + alpha_educ[list(educ)] + alpha_eth[list(eth)] + alpha_age[list(age)] + alpha_male_eth[list(male), list(eth)] + alpha_educ_age[list(educ),list(age)] + alpha_educ_eth[list(educ), list(eth)] + beta_male*male)), obs= obs[ind])
    return y

In [9]:

# Define guide function
guide = AutoDiagonalNormal(model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 12000

# Setup the optimizer
adam_params = {"lr": 0.01}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=3)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X,y)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 107216.9
[500] ELBO: 39786.2
[1000] ELBO: 39696.7
[1500] ELBO: 40093.3
[2000] ELBO: 40287.8
[2500] ELBO: 40059.8
[3000] ELBO: 39689.1
[3500] ELBO: 39816.2
[4000] ELBO: 39911.1
[4500] ELBO: 40493.2
[5000] ELBO: 39611.8
[5500] ELBO: 40043.9
[6000] ELBO: 40792.2
[6500] ELBO: 40718.0
[7000] ELBO: 39702.9
[7500] ELBO: 40905.0
[8000] ELBO: 40209.2
[8500] ELBO: 39642.4
[9000] ELBO: 39760.4
[9500] ELBO: 39889.3
[10000] ELBO: 40436.5
[10500] ELBO: 39733.9
[11000] ELBO: 40257.8
[11500] ELBO: 39554.9


In [11]:
def model2(X, obs=None):
    
    sigmas = pyro.sample("sigmas", dist.HalfCauchy(torch.ones(8)*5))
    gammas = pyro.sample("gammas", dist.Normal(torch.zeros(5), 5.0))
    alpha_educ = pyro.sample("alpha_educ", dist.Normal(torch.zeros(5), sigmas[0]))
    alpha_eth = pyro.sample("alpha_eth", dist.Normal(torch.zeros(4), sigmas[1]))
    alpha_age = pyro.sample("alpha_age", dist.Normal(torch.zeros(6), sigmas[2]))
    alpha_male_eth = pyro.sample("alpha_male_eth", dist.Normal(torch.zeros((2,4)), sigmas[3]))
    alpha_educ_age = pyro.sample("alpha_educ_age", dist.Normal(torch.zeros((5,6)), sigmas[4]))
    alpha_educ_eth = pyro.sample("alpha_educ_eth", dist.Normal(torch.zeros((5,4)), sigmas[5]))
    
    # need the GROUP LEVEL PREDICTORS data for states
    # Temporary solution
    ####################
    alpha_state = pyro.sample("alpha_state", dist.Normal(torch.zeros(50), sigmas[6]))
    ####################
    beta_male = pyro.sample("beta_male", dist.Normal(0.,5.))
    with pyro.plate("data"):
        y = pyro.sample("y",dist.Bernoulli(sigmoid(alpha_state[list(X[:,0])]+alpha_educ[list(X[:,4])] + alpha_eth[list(X[:,1])] + alpha_age[list(X[:,3])] + alpha_male_eth[list(X[:,2]), list(X[:,1])] + alpha_educ_age[list(X[:,4]),list(X[:,3])] + alpha_educ_eth[list(X[:,4]), list(X[:,1])] + beta_male*X[:,2])), obs= obs)
    return y

In [ ]:
# Run inference in Pyro
nuts_kernel = NUTS(model2)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200, num_chains=1)
mcmc.run(X,y)

# Show summary of inference results
mcmc.summary()

Warmup:   3%|▎         | 34/1200 [4:45:25, 1152.94s/it, step size=1.75e-03, acc. prob=0.750]

In [ ]:
pyro.__version__